In [1]:
display("text/html", "<style>.container { width:100% !important; }</style>")

In [2]:
using Pkg;
using Flux
using Images
using FileIO
using Glob
using Mmap
using ImageShow
using Base.Iterators: partition
using Flux: onehotbatch,onecold, crossentropy, throttle
using RandomNumbers
using Plots
using ImageView, Images

Read CIFAR10 data.(https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz)
Data is given in binary format where the first byte is the label of
the first image which is a number between 0 and 9. The next 3072 bytes
are the values of the pixels. The first 1024 are R, the next 1024 are G and
the last 1024 are B.

In [3]:
function loadBatches(path)
    batch = Int.(open(read,path))
    labels = [batch[1+(n-1)*3073] for n in 1:1000]
    pixels = [reshape(batch[2+(n-1)*3073:3073+(n-1)*3073],(32,32,3)) for n in 1:1000]/255
    return (pixels,labels)
end

loadBatches (generic function with 1 method)

In [4]:
# Get file names
path = "./cifar-10-batches-bin/"
trainbatch = readdir(glob"data_batch_*.bin",path)
testbatch = readdir(glob"test_batch.bin",path)


# read files and prepare train and test datasets
for file in trainbatch
    if  file==trainbatch[1]
        global X_t, Y_train = loadBatches(file)
    else
        data = loadBatches(file)
        append!(X_t,data[1])
        append!(Y_train,data[2])
    end
end

In [5]:
# Reshape Xt
X_train =[]
for i in 1:size(X_t)[1]
    append!(X_train,X_t[i])
end
X_train = Float32.(reshape(X_train,32,32,3,:))

X_tt,Y_test = loadBatches(testbatch[1])

X_test =[]
for i in 1:size(X_tt)[1]
    append!(X_test,X_tt[i])
end
X_test = Float32.(reshape(X_test,32,32,3,:))

# One-hot encoding
Y_train = onehotbatch(Y_train,0:9)
Y_test = onehotbatch(Y_test,0:9)

10×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false  false  false   true  false  …  false  false  false  false  false
 false  false  false  false  false     false  false   true  false  false
 false  false  false  false  false     false  false  false  false  false
  true  false  false  false  false      true  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false   true   true  false  false     false   true  false  false   true
 false  false  false  false  false     false  false  false  false  false

In [6]:
X_train[:,:,1,2]

32×32 Array{Float32,2}:
 0.603922  0.54902   0.54902   0.533333  …  0.686275   0.647059   0.639216 
 0.494118  0.568627  0.545098  0.537255     0.611765   0.611765   0.619608 
 0.411765  0.490196  0.45098   0.478431     0.603922   0.623529   0.639216 
 0.4       0.486275  0.576471  0.517647     0.576471   0.513726   0.568627 
 0.490196  0.588235  0.541176  0.592157     0.607843   0.368627   0.168627 
 0.607843  0.596078  0.517647  0.709804  …  0.631373   0.4        0.0745098
 0.67451   0.682353  0.666667  0.796078     0.627451   0.423529   0.0784314
 0.705882  0.698039  0.698039  0.815686     0.654902   0.501961   0.290196 
 0.556863  0.52549   0.670588  0.815686     0.647059   0.603922   0.52549  
 0.435294  0.431373  0.752941  0.796078     0.596078   0.611765   0.466667 
 0.415686  0.521569  0.858824  0.701961  …  0.639216   0.713726   0.431373 
 0.427451  0.639216  0.917647  0.662745     0.643137   0.701961   0.388235 
 0.482353  0.752941  0.898039  0.643137     0.521569   0.490196 

In [7]:
train_idx = partition(1:Int32(length(X_train)/(32*32*3)),10)
test_idx = partition(1:Int32(length(X_test)/(32*32*3)),10)

Base.Iterators.PartitionIterator{UnitRange{Int64}}(1:1000, 10)

In [8]:
function make_partition_index(X,batch_size)
    idx = partition(1:Int32(length(X)/(32*32*3)),batch_size)
    indices = [(minimum(i),maximum(i)) for i in idx]
    return indices
end

make_partition_index (generic function with 1 method)

In [9]:
function make_minibatch(X,Y,batch_size)
    indices = [i for i in make_partition_index(X,batch_size)]
    minibatch_X = [X[:,:,:,indices[i][1]:indices[i][2]] for i in 1:length(indices)]
    minibatch_Y = [Y[indices[i][1]:indices[i][2]] for i in 1:length(indices)]
    dataset = [(minibatch_X[i],minibatch_Y[i]) for i in 1:length(indices)]   
    return dataset
end


make_minibatch (generic function with 1 method)

In [10]:
train_set = make_minibatch(X_train,Y_train,128);
test_set = make_minibatch(X_test,Y_test,1);

In [11]:
# VGG16
model() = Chain(
    # Size 32x32
    Conv((3,3), 3=>64,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(64),
    Dropout(0.3),
    # Size 32x32
    Conv((3,3), 64=>64,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(64),
    # Size 32x32
    x -> MaxPool((2,2))(x),
    # Size 16x16
    Conv((3,3), 64=>128,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(128),
    Dropout(0.3),
    # Size 16x16
    Conv((3,3), 128=>128,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(128),
    x -> MaxPool((2,2))(x),
    # Size 8x8
    Conv((3,3), 128=>256,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(256),
    Dropout(0.4),
    # Size 8x8
    Conv((3,3),256=>256, relu, pad=(1,1), stride=(1,1)),
    BatchNorm(256),
    Dropout(0.4),
    # Size 8x8
    Conv((3,3),256=>256, relu, pad=(1,1), stride=(1,1)),
    BatchNorm(256),
    x -> MaxPool((2,2))(x),
    # Size 4x4
    Conv((3,3), 256=>512,relu, pad=(1,1), stride=(1,1)),
    BatchNorm(512),
    Dropout(0.4),
    # Size 4x4
    Conv((3,3),512=>512, relu, pad=(1,1), stride=(1,1)),
    BatchNorm(512),
    Dropout(0.4),
    # Size 4x4
    Conv((3,3),512=>512, relu, pad=(1,1), stride=(1,1)),
    BatchNorm(512),
    x -> MaxPool((2,2))(x),
    # Size 2x2      
    x -> reshape(x,:,size(x,4)),
    Dense(2048,4096,relu),
    Dropout(0.5),
    Dense(4096,10),
    softmax
)
    

model (generic function with 1 method)

In [12]:
m = model()

Chain(Conv((3, 3), 3=>64, NNlib.relu), BatchNorm(64), Dropout{Float64}(0.3, true), Conv((3, 3), 64=>64, NNlib.relu), BatchNorm(64), getfield(Main, Symbol("##17#22"))(), Conv((3, 3), 64=>128, NNlib.relu), BatchNorm(128), Dropout{Float64}(0.3, true), Conv((3, 3), 128=>128, NNlib.relu), BatchNorm(128), getfield(Main, Symbol("##18#23"))(), Conv((3, 3), 128=>256, NNlib.relu), BatchNorm(256), Dropout{Float64}(0.4, true), Conv((3, 3), 256=>256, NNlib.relu), BatchNorm(256), Dropout{Float64}(0.4, true), Conv((3, 3), 256=>256, NNlib.relu), BatchNorm(256), getfield(Main, Symbol("##19#24"))(), Conv((3, 3), 256=>512, NNlib.relu), BatchNorm(512), Dropout{Float64}(0.4, true), Conv((3, 3), 512=>512, NNlib.relu), BatchNorm(512), Dropout{Float64}(0.4, true), Conv((3, 3), 512=>512, NNlib.relu), BatchNorm(512), getfield(Main, Symbol("##20#25"))(), getfield(Main, Symbol("##21#26"))(), Dense(2048, 4096, NNlib.relu), Dropout{Float64}(0.5, true), Dense(4096, 10), NNlib.softmax)

In [13]:
loss(x,y) = crossentropy(m(x),y)

loss (generic function with 1 method)

In [14]:
accuracy(x,y) = mean(onecold(m(x),1:10) .== onecold(y,1:10))

accuracy (generic function with 1 method)

In [15]:
evalcb = throttle(()-> @show(accuracy(X_test,Y_test)),10)

(::getfield(Flux, Symbol("#throttled#18")){getfield(Flux, Symbol("##throttled#10#14")){Bool,Bool,getfield(Main, Symbol("##27#28")),Int64}}) (generic function with 1 method)

In [16]:
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [17]:
Flux.train!(loss, params(m),train_set,opt)

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size")

In [18]:
size(train_set)

(40,)

In [19]:
# 469-element Array{Tuple{Array{Float32,4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:


In [20]:
typeof(train_set)

Array{Tuple{Array{Float32,4},Array{Bool,1}},1}